In [1]:
using CSV
using DataFrames
using PrettyTables
using StatsBase
include("./advgames_network.jl")

initialize (generic function with 1 method)

In [5]:
# Define special data to record

adv_coop(player) = player.adv_type == :C
adv_defect(player) = player.adv_type == :D
should_c(player) = est_move_coop(player) > est_move_defect(player)
should_d(player) = est_move_coop(player) < est_move_defect(player)
should_adv_c(player) = est_adv_coop(player) > est_adv_defect(player)
should_adv_d(player) = est_adv_coop(player) < est_adv_defect(player)

function cum_avg(a::Vector{Float64})  # helper function that returns vector of cumulative averages of vector a
      ca=Float64[]    
      for i in 1:length(a)
          push!( ca, sum(first(a,i))/i )
      end
return ca
end


#individual_incls(model) = [pr_coop(p) for p in allagents(model)]
avg_incl(model) = sum([pr_coop(p) for p in allagents(model)]) / nagents(model)  
 
pr_max_coop(model) = sum([pr_coop(p) == 1.0 for p in allagents(model)]) / nagents(model)
pr_max_defect(model) = sum([pr_coop(p) == 0.0 for p in allagents(model)]) / nagents(model)

cooperate_state(model) = pr_max_coop(model)>.75
defect_state(model) = pr_max_defect(model) >.75

pr_adv_coop(model) = sum([advocate(p) == :C for p in allagents(model)]) / nagents(model)
pr_adv_defect(model) = sum([advocate(p) == :D for p in allagents(model)]) / nagents(model)

pr_hyp_adv_c_play_d(model) = sum([advocate(p) == :C && pr_coop(p) <= 0.15 for p in allagents(model)]) / nagents(model)
pr_hyp_adv_d_play_c(model) = sum([advocate(p) == :D && pr_coop(p) >= 0.85 for p in allagents(model)]) / nagents(model)

avg_pr_max_coop(model) = mean([pr_coop(p) for p in allagents(model)])  >= 0.85
avg_pr_max_defect(model) = mean([pr_coop(p) for p in allagents(model)])  <= 0.15

pr_mistaken_adv(model) = sum([(advocate(p) == :C && est_move_coop(p) < est_move_defect(p)) ||
                              (advocate(p) == :D && est_move_coop(p) > est_move_defect(p))
                            for p in allagents(model)]) / nagents(model)

proportion_should_c(model) = sum([should_c(p) for p in allagents(model)]) / nagents(model)
proportion_should_d(model) = sum([should_d(p) for p in allagents(model)]) / nagents(model)
proportion_should_adv_c(model) = sum([should_adv_c(p) for p in allagents(model)]) / nagents(model)
proportion_should_adv_d(model) = sum([should_adv_d(p) for p in allagents(model)]) / nagents(model)

cc(model) = float(model.payouts[:C, :C])
cd(model) = float(model.payouts[:C, :D])
dc(model) = float(model.payouts[:D, :C])
dd(model) = float(model.payouts[:D, :D])

is_pd(model) = dc(model) > cc(model) > dd(model) > cd(model)
is_chicken(model) = dc(model) > cc(model) && cd(model) > dd(model)
is_sh(model) = cc(model) > dc(model) && dd(model) > cd(model) && cc(model) > dd(model)
is_c_dom(model) = cc(model) > dc(model) > cd(model) > dd(model)
is_d_dom(model) = dd(model) > cd(model) > dc(model) > cc(model)

function game_type(model) 
    if is_pd(model)
        return "PD"
    elseif is_chicken(model)
        return "CH"
    elseif is_sh(model)
        return "SH"
    elseif is_c_dom(model)
        return "Cdom"
    elseif is_d_dom(model)
        return "Ddom"
    else 
        return "Other"
    end
end


game_type (generic function with 1 method)

In [6]:
model, parameters = initialize(    
    pr_cooperator = 0.5, 
    init_pr_coop_for_cooperator = 0.75, 
    pr_defector = 0.5, 
    init_pr_coop_for_defector = 0.25, 
    pr_neutral = 0.0,
    dims = (10, 10),
    influence_type = :global,
    interaction_type = :global,
    rstp = nothing,
    payouts = Dict(
        (:C, :C) => 3, 
        (:C, :D) => 0, 
        (:D, :C) => 5, 
        (:D, :D) => 1), 
    adv_reassess_frequency = 0.03, 
    move_reassess_frequency = 0.1,
    max_pos_pressure = 6, 
    max_neg_pressure = 6,
    tolerance = 0.05,
)

pretty_table(parameters, show_subheader=false)
open("initial_experiment_parameters_network_global_global.txt", "w") do io
    pretty_table(io, parameters, show_subheader=false)
end

┌──────────────────────────┬──────────────┐
│                     Keys │       Values │
├──────────────────────────┼──────────────┤
│         interaction_type │       global │
│             base_payouts │ [5, 3, 1, 0] │
│                tolerance │         0.05 │
│            pr_cooperator │          0.5 │
│               pr_neutral │          0.0 │
│               population │          100 │
│    max_pos_neg_pressures │       [6, 6] │
│       a_m_mutation_rates │ [0.02, 0.15] │
│   a_m_weights_of_present │   [0.3, 0.2] │
│ a_m_reassess_frequencies │  [0.03, 0.1] │
│              pr_defector │          0.5 │
│           influence_type │       global │
│              m_increment │         0.15 │
└──────────────────────────┴──────────────┘


In [7]:
numrounds=100_000

adata = [pr_coop, adv_coop, est_move_coop, est_move_defect, est_adv_coop, est_adv_defect]

# model data to record
mdata = [
    cc, 
    cd, 
    dc, 
    dd, 
    game_type,
    avg_incl,
    pr_adv_coop,
    pr_hyp_adv_c_play_d, 
    pr_hyp_adv_d_play_c, 
    ]

adf, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

# write mdf to csv
CSV.write("model_data_init_network_global_global.csv", mdf)
CSV.write("agent_data_init_network_global_global.csv", adf)

"agent_data_init_network_global_global.csv"

In [8]:
model, parameters = initialize(    
    pr_cooperator = 0.5, 
    init_pr_coop_for_cooperator = 0.75, 
    pr_defector = 0.5, 
    init_pr_coop_for_defector = 0.25, 
    pr_neutral = 0.0,
    dims = (10, 10),
    influence_type = :local,
    interaction_type = :global,
    rstp = nothing,
    payouts = Dict(
        (:C, :C) => 3, 
        (:C, :D) => 0, 
        (:D, :C) => 5, 
        (:D, :D) => 1), 
    adv_reassess_frequency = 0.03, 
    move_reassess_frequency = 0.1,
    max_pos_pressure = 6, 
    max_neg_pressure = 6,
    tolerance = 0.05,
)

pretty_table(parameters, show_subheader=false)
open("initial_experiment_parameters_network_local_global.txt", "w") do io
    pretty_table(io, parameters, show_subheader=false)
end

┌──────────────────────────┬──────────────┐
│                     Keys │       Values │
├──────────────────────────┼──────────────┤
│         interaction_type │       global │
│             base_payouts │ [5, 3, 1, 0] │
│                tolerance │         0.05 │
│            pr_cooperator │          0.5 │
│               pr_neutral │          0.0 │
│               population │          100 │
│    max_pos_neg_pressures │       [6, 6] │
│       a_m_mutation_rates │ [0.02, 0.15] │
│   a_m_weights_of_present │   [0.3, 0.2] │
│ a_m_reassess_frequencies │  [0.03, 0.1] │
│              pr_defector │          0.5 │
│           influence_type │        local │
│              m_increment │         0.15 │
└──────────────────────────┴──────────────┘


In [9]:
numrounds=100_000

adata = [pr_coop, adv_coop, est_move_coop, est_move_defect, est_adv_coop, est_adv_defect]

# model data to record
mdata = [
    cc, 
    cd, 
    dc, 
    dd, 
    game_type,
    avg_incl,
    pr_adv_coop,
    pr_hyp_adv_c_play_d, 
    pr_hyp_adv_d_play_c, 
    ]

adf, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

# write mdf to csv
CSV.write("model_data_init_network_local_global.csv", mdf)
CSV.write("agent_data_init_network_local_global.csv", adf)

"agent_data_init_network_local_global.csv"

In [10]:
model, parameters = initialize(    
    pr_cooperator = 0.5, 
    init_pr_coop_for_cooperator = 0.75, 
    pr_defector = 0.5, 
    init_pr_coop_for_defector = 0.25, 
    pr_neutral = 0.0,
    dims = (10, 10),
    influence_type = :global,
    interaction_type = :local,
    rstp = nothing,
    payouts = Dict(
        (:C, :C) => 3, 
        (:C, :D) => 0, 
        (:D, :C) => 5, 
        (:D, :D) => 1), 
    adv_reassess_frequency = 0.03, 
    move_reassess_frequency = 0.1,
    max_pos_pressure = 6, 
    max_neg_pressure = 6,
    tolerance = 0.05,
)

pretty_table(parameters, show_subheader=false)
open("initial_experiment_parameters_network_global_local.txt", "w") do io
    pretty_table(io, parameters, show_subheader=false)
end

┌──────────────────────────┬──────────────┐
│                     Keys │       Values │
├──────────────────────────┼──────────────┤
│         interaction_type │        local │
│             base_payouts │ [5, 3, 1, 0] │
│                tolerance │         0.05 │
│            pr_cooperator │          0.5 │
│               pr_neutral │          0.0 │
│               population │          100 │
│    max_pos_neg_pressures │       [6, 6] │
│       a_m_mutation_rates │ [0.02, 0.15] │
│   a_m_weights_of_present │   [0.3, 0.2] │
│ a_m_reassess_frequencies │  [0.03, 0.1] │
│              pr_defector │          0.5 │
│           influence_type │       global │
│              m_increment │         0.15 │
└──────────────────────────┴──────────────┘


In [11]:
numrounds=100_000

adata = [pr_coop, adv_coop, est_move_coop, est_move_defect, est_adv_coop, est_adv_defect]

# model data to record
mdata = [
    cc, 
    cd, 
    dc, 
    dd, 
    game_type,
    avg_incl,
    pr_adv_coop,
    pr_hyp_adv_c_play_d, 
    pr_hyp_adv_d_play_c, 
    ]

adf, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

# write mdf to csv
CSV.write("model_data_init_network_global_local.csv", mdf)
CSV.write("agent_data_init_network_global_local.csv", adf)

"agent_data_init_network_global_local.csv"

In [12]:
model, parameters = initialize(    
    pr_cooperator = 0.5, 
    init_pr_coop_for_cooperator = 0.75, 
    pr_defector = 0.5, 
    init_pr_coop_for_defector = 0.25, 
    pr_neutral = 0.0,
    dims = (10, 10),
    influence_type = :local,
    interaction_type = :local,
    rstp = nothing,
    payouts = Dict(
        (:C, :C) => 3, 
        (:C, :D) => 0, 
        (:D, :C) => 5, 
        (:D, :D) => 1), 
    adv_reassess_frequency = 0.03, 
    move_reassess_frequency = 0.1,
    max_pos_pressure = 6, 
    max_neg_pressure = 6,
    tolerance = 0.05,
)

pretty_table(parameters, show_subheader=false)
open("initial_experiment_parameters_network_local_local.txt", "w") do io
    pretty_table(io, parameters, show_subheader=false)
end

┌──────────────────────────┬──────────────┐
│                     Keys │       Values │
├──────────────────────────┼──────────────┤
│         interaction_type │        local │
│             base_payouts │ [5, 3, 1, 0] │
│                tolerance │         0.05 │
│            pr_cooperator │          0.5 │
│               pr_neutral │          0.0 │
│               population │          100 │
│    max_pos_neg_pressures │       [6, 6] │
│       a_m_mutation_rates │ [0.02, 0.15] │
│   a_m_weights_of_present │   [0.3, 0.2] │
│ a_m_reassess_frequencies │  [0.03, 0.1] │
│              pr_defector │          0.5 │
│           influence_type │        local │
│              m_increment │         0.15 │
└──────────────────────────┴──────────────┘


In [13]:
numrounds=100_000

adata = [pr_coop, adv_coop, est_move_coop, est_move_defect, est_adv_coop, est_adv_defect]

# model data to record
mdata = [
    cc, 
    cd, 
    dc, 
    dd, 
    game_type,
    avg_incl,
    pr_adv_coop,
    pr_hyp_adv_c_play_d, 
    pr_hyp_adv_d_play_c, 
    ]

adf, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)

# write mdf to csv
CSV.write("model_data_init_network_local_local.csv", mdf)
CSV.write("agent_data_init_network_local_local.csv", adf)

"agent_data_init_network_local_local.csv"